# Testing Individual components of the FV HE scheme

In [1]:
from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encrypter import Encrypter
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/ravikantsingh/Desktop/PySyft/venv/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


## Generates coeff_modulus using Polymodulus degree and bit-length parameters.
Generates random integer of the given bit length for using as coeff_modulus

In [2]:
params = EncryptionParams()
params.poly_modulus_degree = 1024
params.plain_modulus = 64
cm = CoeffModulus()
params.coeff_modulus = cm.create(4096, [30, 50, 60])
print("coeff_modulus: ", params.coeff_modulus)

coeff_modulus:  [1073692673, 1125899906826241, 1152921504606830593]


## Use standard values of coeff_modulus for the given Poly Modulus degree and security_level.
Need some refractoring and integration of code.

In [3]:
# params_standard = EncryptionParams()
# params_standard.poly_modulus_degree = 1024
# params_standard.plain_modulus = 64
# cm_standard = CoeffModulus()
# params.coeff_modulus = cm_standard.bfv_default(poly_modulus, seq_level)
# print("coeff_modulus: ", params.coeff_modulus)

## Keygeneration

In [4]:
ctx = Context(params)
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [5]:
print('secret key values : ', sk.data)
print(sk)

secret key values :  tensor([         1073692672,                   0,          1073692672,
         ...,                   1,                   1,
        1152921504606830592])


In [6]:
print('public key values : ', pk.data)
print(pk)

public key values :  [tensor([         626823032,         1073692671,          430622948,
         ...,  43901817688109317, 582861923654278665,
        539320566441181175]), tensor([          626823040,           781198415,           430622943,
         ..., 1109019686918721280,  570059580952551925,
         168354953588013859])]


## Integer Encoder
Encodes Integer values to Plaintext object

In [13]:
int_encoder = IntegerEncoder(ctx)
plaintext = int_encoder.encode(6)
print(plaintext)
print('plaintext data',plaintext.data)

plaintext data [0, 1, 1]


### Decodes back to Integer

In [8]:
print(int_encoder.decode(plaintext))

6


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [14]:
encrypter = Encrypter(ctx, pk)
ciphertext = encrypter.encrypt(plaintext)
print(ciphertext)
print('ciphertext data :', ciphertext.data)

ciphertext data : [tensor([          480422528,                   2,                   2,
         ..., 1152921504606830591,                   0,
        1692242071048011761]), tensor([          446869636,          1366186922,                   1,
         ..., 1152921504606830591, 1152921504606830591,
         168354953588013860])]


Encrypt Plaintext to ciphertext using secret_key

In [10]:
# TODO Need some changes.

# encrypter_symmetric = Encrypter(ctx, sk)
# ciphertext_symmetric = encrypter_symmetric.encrypt(plaintext)
# print(ciphertext_symmetric)